# This notebook imports the motion data collected on board the R/V Akademik Tryoshnikov during the Antarctic Circumnavigation Expedition (ACE). 

Import required packages

In [ ]:
import csv
import os
import pandas
import datetime
import time
import io
import numpy as np

Set up pandas display

In [ ]:
pandas.set_option('display.max_columns', 100)
pandas.set_option('display.max_rows', 20000)

Set up the hard-coded variables to import the data.

In [ ]:
test_input_data_folder = "/home/jen/projects/ace_data_management/ship_data/motion_data/test"
output_data_folder = "/home/jen/projects/ace_data_management/wip/motion_data/"

### List data files

Get the set of raw motion data files in a list.

In [ ]:
def get_input_txt_files(input_data_folder):
    """Create a list of files in a directory and put them into a list."""
    
    list_data_files = []
    
    os.chdir(input_data_folder)
    directory_path = os.getcwd()
    
    for filename in os.listdir(input_data_folder):
        if filename.endswith(".txt"):
            fullpath = directory_path + "/" + filename
            list_data_files.append(fullpath)
    
    return list_data_files

### Check file headers

Check that all the files have the same header so that when creating the pandas data frame, the header can always be excluded. 

The header consists of 5 lines - an example below: 

Serial n° Hydrins :     PH-497

Created the :   07/01/2017              Time :  11h 10m 39.822s

Sample period : 1000 ms

Pc - HH:MM:SS.SSS       Hydrins - HH:MM:SS.SSS  Heading (°)     Roll (°)        Pitch (°)       Heading std. dev. (°)   Roll std. dev. (°)      Pitch std. dev. (°)     North speed (m/s)       East speed (m/s)        Vert. speed (m/s)       Speed norm (knots)      North speed std. dev. (m/s)     East speed std. dev. (m/s)      Vert. speed std. dev. (m/s)     Latitude (°)    Longitude (°)   Altitude (m)    Latitude std. dev. (m)  Longitude std. dev. (m) Altitude std. dev. (m)  Zone I  Zone C  UTM North (m)   UTM East (m)    UTM altitude  (m)       High level status       System status 1 System status 2 Algo status 1   Algo status 2   GPS - Latitude (°)      GPS - Longitude (°)     GPS - Altitude (m)      GPS - Mode      GPS - Time      Manual GPS - Latitude (°)       Manual GPS - Longitude (°)      Manual GPS - Altitude (m)       Manual GPS - Latitude std. dev. Manual GPS - Longitude std. dev.        Manual GPS - Altitude std. dev.

Note that we will need the date from the second line in the header and the column header will be checked and set as the column headers in the data frame. The length of the header in the files is not the same as the number of data columns so this also needs to be checked. 

The header contains 43 "columns", the last one of which is empty. Whereas each line of data contains 45 columns, also with the last one empty. Therefore there are two columns which are not labelled and three in total which do not have a header (including the unnamed ones). These unnamed columns have been labelled as unknown1, unknown2 and unknown3. 

The expected header is defined in order to compare that within each file (note that this ignores the first four lines).

In [ ]:
expected_header = ['Pc - HH:MM:SS.SSS', 'Hydrins - HH:MM:SS.SSS', 'Heading (°)', 'Roll (°)', 'Pitch (°)', 'Heading std. dev. (°)', 'Roll std. dev. (°)', 'Pitch std. dev. (°)', 'North speed (m/s)', 'East speed (m/s)', 'Vert. speed (m/s)', 'Speed norm (knots)', 'North speed std. dev. (m/s)', 'East speed std. dev. (m/s)', 'Vert. speed std. dev. (m/s)', 'Latitude (°)', 'Longitude (°)', 'Altitude (m)', 'Latitude std. dev. (m)', 'Longitude std. dev. (m)', 'Altitude std. dev. (m)', 'Zone I', 'Zone C', 'UTM North (m)', 'UTM East (m)', 'UTM altitude  (m)', 'High level status', 'System status 1', 'System status 2', 'Algo status 1', 'Algo status 2', 'GPS - Latitude (°)', 'GPS - Longitude (°)', 'GPS - Altitude (m)', 'GPS - Mode', 'GPS - Time', 'Manual GPS - Latitude (°)', 'Manual GPS - Longitude (°)', 'Manual GPS - Altitude (m)', 'Manual GPS - Latitude std. dev.', 'Manual GPS - Longitude std. dev.', 'Manual GPS - Altitude std. dev.', '']

In [ ]:
len(expected_header)

In [ ]:
def check_file_header(list_data_files): 
    """Check that the header for each of the files in the list is as expected. If it is correct, add one to the number of correct headers that is output and if incorrect, output the filename and a copy of the header line. Also add one to the number of incorrect headers."""
    
    correct_headers = 0
    incorrect_headers = 0
    total_number_files = len(list_data_files)
    
    total_number_headers = 0
    for file in list_data_files:
        total_number_headers += 1
        print("Checking the header of file", total_number_headers, "out of", total_number_files)
        
        with open(file, 'r', encoding="ISO-8859-1") as csvfile: # encoding that of original files - required because of degrees characters
            contents = csv.reader(csvfile, delimiter='\t')
        
            line_number = 0
        
            for line in contents:
                if line_number == 4:
                    if line != expected_header:
                        print("Wrong header: ", file, "  ", line)
                        incorrect_headers += 1
                    else:
                        correct_headers += 1
                
                line_number += 1
    
    total_no_files = correct_headers + incorrect_headers
    
    print("Correct headers: ", correct_headers)
    print("Incorrect headers: ", incorrect_headers)
    print("Total number of files: ", total_no_files)

### Get the date of the data collection and read the data into a list

Read header of file to extract the date. This is important because otherwise, when the data are read into a data frame, they will not have a date associated with them.

Note that the line within the header that contains the date is, for example: 

Created the : 07/01/2017 Time : 11h 10m 39.822s

However not all the data may be collected on that particular date. Each data file contains 1000 ms of data, so if the file begins just before midnight then it is possible that it may end with data from the following day. This needs to be taken into account when assigning the date to the data. 

So far only the PC date has been added as a separate field (pc_date_utc). it is possible that the other time fields have a different date associated with them (Hydrins and GPS). 

In [ ]:
def read_motion_file_date(filename):
    """Read a file and get the date from the second line in the header. Output the date in the given format."""
    
    with open(filename, 'r', encoding="ISO-8859-1") as data_file: # need encoding because of degree characters
        data_file.readline() # skips first line
        date_line = data_file.readline()
        
        date = date_line.split("\t")[1]
        
        return datetime.datetime.strptime(date, "%d/%m/%Y")
    
    
def optimise_line(line):
    """Convert the values in a line of data that look like numbers, to floats (to optimise the memory usage and make the next stage more efficient). If the value is not a number, then leave it in its original format."""
    for i, value in enumerate(line):
        try:
            line[i] = float(line[i])
        except ValueError:
            pass  
        
    
#def data_to_list(list_data_files, rows_of_data):
#    """Read files from a list of files, get the date from each file, then append the date to each line within the file as the line of data is read into a list. Output a list of data from all of the files."""
#    
#    total_number_files = len(list_data_files)
#    
#    total_rows = 0
#    number_files = 0
#    for file in list_data_files:
#        number_files += 1
#        print("Processing file", number_files, " of ", total_number_files)
#        
#        date_of_file = read_motion_file_date(file)
#    
#        with open(file, 'r', encoding="ISO-8859-1") as data_file:
#            contents = csv.reader(data_file, delimiter='\t')
#            for i in range(5): 
#                next(contents)
#                    
#            previous_time_pc = None
#            row_count = 0
#            for line in contents:
#
#                time_now_pc = time.strptime(line[0], "%H:%M:%S.%f")
#
#                if previous_time_pc is None: 
#                    current_date = date_of_file
#                elif time_now_pc < previous_time_pc:
#                    current_date = current_date + datetime.timedelta(days=1)
#                
#                line.insert(0, current_date.strftime("%Y-%m-%d"))
#                previous_time_pc = time_now_pc
#                optimise_line(line)
#                rows_of_data.append(line) 
#
#                row_count += 1
#            
#            print(file, "contains a total number of rows:", row_count)
#        
#        total_rows += row_count
#    #print("\nTotal number of rows expected in dataframe:", total_rows) # this is taking too long
#                
#    return rows_of_data   

def data_to_list(filename, rows_of_data):
    """Read files from a list of files, get the date from each file, then append the date to each line within the file as the line of data is read into a list. Output a list of data from all of the files."""

    date_of_file = read_motion_file_date(filename)

    with open(filename, 'r', encoding="ISO-8859-1") as data_file:
        contents = csv.reader(data_file, delimiter='\t')
        for i in range(5):
            next(contents)

        pc_previous_time = None
        row_count = 0
        for line in contents:

            pc_time_now = time.strptime(line[0], "%H:%M:%S.%f")
            #print("PC", line[0], pc_time_now)
            hydrins_time_now = time.strptime(line[1], "%H:%M:%S.%f")
            #print("Hydrins", line[1], hydrins_time_now)
            gps_time_now = time.strptime(line[35], "%H:%M:%S.%f")
            #print("GPS", line[35], gps_time_now)
            
            if pc_previous_time is None:
                pc_current_date = date_of_file
            elif pc_time_now < pc_previous_time:
                pc_current_date = pc_current_date + datetime.timedelta(days=1)
            #print("pc current date", pc_current_date)

            line.insert(0, pc_current_date.strftime("%Y-%m-%d"))
            
            if hydrins_time_now >= time.strptime("23:00:00.0", "%H:%M:%S.%f") and pc_time_now <= time.strptime("01:00:00.0", "%H:%M:%S.%f"):
                hydrins_current_date = pc_current_date - datetime.timedelta(days=1)
            elif pc_time_now >= time.strptime("23:00:00.0", "%H:%M:%S.%f") and hydrins_time_now <= time.strptime("01:00:00.0", "%H:%M:%S.%f"):
                hydrins_current_date = pc_current_date + datetime.timedelta(days=1)
            else:
                hydrins_current_date = pc_current_date
                
                #hydrins_time_now_t = time.mktime(hydrins_time_now)
                #pc_time_now_t = time.mktime(pc_time_now)
                #if abs(pc_time_now_t-hydrins_time_now_t) > 7200:
                #    raise "Failed"
                # ime.strptime(pc_time_now) - time.strptime(hydrins_time_now)) <= time.strptime("02:00:00.0", "%H:%M:%S.%f"):
                #hydrins_current_date = pc_current_date

            #print("hydrins current date", hydrins_current_date)
                
            line.insert(2, hydrins_current_date.strftime("%Y-%m-%d"))
            
            if gps_time_now >= time.strptime("23:00:00.0", "%H:%M:%S.%f") and pc_time_now <= time.strptime("01:00:00.0", "%H:%M:%S.%f"):
                gps_current_date = pc_current_date - datetime.timedelta(days=1)
            elif pc_time_now >= time.strptime("23:00:00.0", "%H:%M:%S.%f") and gps_time_now <= time.strptime("01:00:00.0", "%H:%M:%S.%f"):
                gps_current_date = pc_current_date + datetime.timedelta(days=1)
            else:
                gps_current_date = pc_current_date
            
            #print("gps current date", gps_current_date)

            line.insert(37, gps_current_date.strftime("%Y-%m-%d"))
            #print(line)
            pc_previous_time = pc_time_now
            optimise_line(line)
            rows_of_data.append(line)

            row_count += 1

        print(filename, "contains a total number of rows:", row_count)

    return rows_of_data    

### Define the column headers

Get the header for the columns so that this can be assigned to the data frame. 

Currently this is listed in a csv file so that it can be changed and reimported as necessary. 

The data contains more columns than are listed in the header in the data files, so we need to ensure that these are assigned correctly in the pandas data frame. 

In [ ]:
def define_column_headers(header_file):
    """Import a list of column headers from a csv file and output them as a list to be ready to be set as the column headers for a pandas data frame."""

    header = []

    with open(header_file) as headerfile:
        contents = csv.reader(headerfile)
        header_list = list(contents)
    
        for item in header_list: 
            header.append(item[0])
            
    return header

In [ ]:
header_file = "/home/jen/projects/ace_data_management/wip/motion_data/file_header.csv"

header = define_column_headers(header_file)

### Create data frame and import data

Import all of the data into a data frame using pandas, which has the column headers as defined from the csv file of column headers.

The date needs to be extracted from the file header depending on the time of the data point, and inserted as an additional column in the data frame. 

Data will be put into a list first, so that the date can be added, then this list of data will be added to the pandas data frame.

In [ ]:
def data_to_dataframe(rows_of_data, dataframe, header):
    """Import a list of data into a pandas dataframe and use a list of column names as the header."""
    
    dataframe = dataframe.append(pandas.DataFrame(rows_of_data, columns = header), ignore_index = True)
    
    return dataframe

Optimise memory usage in the dataframe by converting float64 to float32 (uses less bytes per digit).

In [ ]:
# The code below was taken from https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65 and is used to convert the datatype to one that uses less memory.

def reduce_memory_usage(props):
    """Takes a dataframe and converts the data type of each float to float64 (oroignally did it to float32 but wanted more dp for lat and lon, reducing the memory usage."""
    
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float64) # changed this from float32 to avoid losing the accuracy of the latitude and longitude
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
            
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

### Output the data into csv files with one file per date.

In [ ]:
def output_daily_files(dataframe, output_data_folder):
    """Output a pandas dataframe to a files where each file contains one day's worth of data. Output the data file to the output data folder."""
    
    output_filename_base = 'ace_hydrins_'

    date_group = dataframe.groupby('pc_date_utc')
    print("Aggregated groups by date with counts:")
    print(dataframe.groupby('pc_date_utc').size())
    print("\nTotal number of records:")
    print(dataframe.groupby('pc_date_utc').size().sum())
    
    for date in date_group.groups:
        date_formatted = datetime.datetime.strptime(date, "%Y-%m-%d")    

        date_string = date_formatted.strftime('%Y%m%d')
       
        output_filename = output_data_folder + output_filename_base + date_string + ".csv"
        
        date_group.get_group(date).to_csv(output_filename, sep=",", header=True, index=False)
        print(date, "file created") # TODO put a better check here that the file exists

### Set up a test list of files and test the import. 

In [ ]:
test_list_data_files = ['/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_300.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_301.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_302.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_303.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_304.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_305.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_306.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_307.txt', '/media/jen/SAMSUNG/motion_data/ACE_Bremen-2_308.txt']

In [ ]:
#print("Checking headers of files")
#print("\n")

#check_file_header(test_list_data_files)

#print("Reading files and creating data frame")

#motiondf = pandas.DataFrame(columns = header)

#rows_of_data = list()    

#rows_of_data = data_to_list(test_list_data_files, rows_of_data)

#motiondf = data_to_dataframe(rows_of_data, motiondf, header)

#print("The dataframe has", len(motiondf), "rows")

Preview the dataframe

In [ ]:
#motiondf.iloc[:5]

Test outputting the files (one per day)

In [ ]:
#output_daily_files(motiondf, output_data_folder)

## Get the list of data files and import them into the dataframe. 

Create a list of data files.

In [ ]:
list_motion_data_files = get_input_txt_files(test_input_data_folder)
print(len(list_motion_data_files))

Check the headers of the files.

In [ ]:
print("Checking headers of files")
print("\n")

t1_start = time.perf_counter()
t2_start = time.process_time()

check_file_header(list_motion_data_files)

t1_stop = time.perf_counter()
t2_stop = time.process_time()

elapsed_time_headers = t1_stop - t1_start
cpu_processing_time_header = t2_stop - t2_start

print("Elapsed time headers:", elapsed_time_headers, "secs")
print("CPU processing time headers:", cpu_processing_time_header, "secs")

Read the data into a list, adding the date to the data in the process.

In [ ]:
#print("Reading files and creating data frame")
#motiondf = pandas.DataFrame(columns = header)

#rows_of_data = list()    

#t1_start = time.perf_counter()
#t2_start = time.process_time()

#rows_of_data = data_to_list(list_motion_data_files, rows_of_data)

#t1_stop = time.perf_counter()
#t2_stop = time.process_time()

#elapsed_time_date_to_rows = t1_stop - t1_start
#cpu_processing_time_date_to_rows = t2_stop - t2_start

#print("Elapsed time date to rows:", elapsed_time_date_to_rows, "secs")
#print("CPU processing time date to rows:", cpu_processing_time_date_to_rows, "secs")

In [ ]:
len(list_motion_data_files)

In [ ]:
rows_of_data = list()
motiondf = pandas.DataFrame(columns = header)

t1_start = time.perf_counter()
t2_start = time.process_time()

no_files_processed = 0
for filename in list_motion_data_files:
    print(filename)
    total_no_files = len(list_motion_data_files)
    
    data_to_list(filename, rows_of_data)
    motiondf = data_to_dataframe(rows_of_data, motiondf, header)
    rows_of_data = list()
    
    reduce_memory_usage(motiondf)
    
    no_files_processed += 1
    print("Processed", no_files_processed, "out of", total_no_files)
    
t1_stop = time.perf_counter()
t2_stop = time.process_time()
    
elapsed_time = t1_stop - t1_start
cpu_processing_time = t2_stop - t2_start

print("Elapsed time:", elapsed_time, "secs")
print("CPU processing time:", cpu_processing_time, "secs")

print("\n******************************\n")
motiondf.iloc[:5]

RUN TO HERE then pickle then output data files

Review the list of data.

In [ ]:
#print(len(rows_of_data))
#type(rows_of_data)
#print(rows_of_data[0:5])
#print(len(rows_of_data[1]))

Convert the list of data to a pandas dataframe.

In [ ]:
#t1_start = time.perf_counter()
#t2_start = time.process_time()

#motiondf = data_to_dataframe(rows_of_data, motiondf, header)

#print("The dataframe has", len(motiondf), "rows\n")

#t1_stop = time.perf_counter()
#t2_stop = time.process_time()

#elapsed_time_dataframe = t1_stop - t1_start
#cpu_processing_time_dataframe = t2_stop - t2_start

#print("Elapsed time dataframe:", elapsed_time_dataframe, "secs")
#print("CPU processing time dataframe:", cpu_processing_time_dataframe, "secs")

Preview the data frame.

In [ ]:
#motiondf.iloc[:5]
#motiondf.info()

Optimise the memory usage of the dataframe.

In [ ]:
#motiondf, NAlist = reduce_memory_usage(motiondf)

In [ ]:
#import gc
#import sys
#sys.getrefcount(rows_of_data)
#rows_of_data=None
#gc.collect()

Pickle the dataframe (output it to a file to remove it from the memory) and check the memory usage again.

In [ ]:
motiondf.to_pickle(output_data_folder + "motiondf.pkl")
#motiondf.memory_usage().sum()

In [ ]:
motiondf.reset_index(drop=True, inplace=True)
motiondf.to_pickle(output_data_folder + "motiondf_noindex.pkl")

Output the data files from the dataframe into daily files. 

In [ ]:
motiondf = pandas.read_pickle(output_data_folder + 'motiondf_noindex.pkl')

t1_start = time.perf_counter()
t2_start = time.process_time()

output_daily_files(motiondf, output_data_folder)

t1_stop = time.perf_counter()
t2_stop = time.process_time()

elapsed_time_file_output = t1_stop - t1_start
cpu_processing_time_file_output = t2_stop - t2_start

print("Elapsed time file output:", elapsed_time_file_output, "secs")
print("CPU processing time file output:", cpu_processing_time_file_output, "secs")

### Check that the output files have the same number of data rows as the dataframe had per day.

In [ ]:
data_folder = "/home/jen/projects/ace_data_management/wip/motion_data"

Get a list of the output files to check (named ace_hydrins_yyyymmdd.csv)

In [ ]:
def get_input_files(input_data_folder):
    
    list_data_files = []
    
    os.chdir(input_data_folder)
    directory_path = os.getcwd()
    
    for filename in os.listdir(input_data_folder):
        if filename.startswith("ace_hydrins_"):
            fullpath = directory_path + "/" + filename
            list_data_files.append(fullpath)
    
    return list_data_files

Count the number of rows in each file, ignoring the header and output with the date and number of rows

In [ ]:
def check_rows_in_file(list_data_files):

    total_rows = 0
    for filepath in list_data_files:
        filename = os.path.basename(filepath)
        filedate = (filename.split('_')[-1]).split('.')[0] 

        with open(filepath, 'r') as csvfile:
            contents = csv.reader(csvfile)
            next(contents)

            row_count = 0
            for line in contents:
                row_count += 1

            print(filedate, " ", row_count)
        
        total_rows += row_count
    
    print("Total number of rows in files: ", total_rows)

In [ ]:
list_data_files_to_check = get_input_files(data_folder)

check_rows_in_file(list_data_files_to_check)